In [ ]:
import numpy as np
from pyqubo import Binary
from pyqubo import Array
import neal
import networkx as nx
import matplotlib.pyplot as plt
import math 
import random
from collections import Counter

In [ ]:
def dist(pos,i,j):
    d = (pos[i][0]-pos[j][0])**2+(pos[i][1]-pos[j][1])**2
    return math.sqrt(d)

In [ ]:
N=20

GR=nx.Graph();
for i in range(N):
    GR.add_node(i)
pos=nx.random_layout(GR)
nx.draw(GR,pos=pos,with_labels = True)
ax = plt.gca()
R = []
for i in range(N):
    center = pos[i]
    R.append(random.uniform(0.1,0.3))
    c = plt.Circle(center, R[i])
    ax.add_patch(c)
    
Area = np.zeros(N, dtype=float)    
for i in range(N):
    Area[i] = R[i]*R[i]*math.pi

plt.axis('equal')
plt.show()

In [ ]:
Rho = np.zeros((N,N), dtype=float)
for i in range(N):
    for j in range(i+1,N):
        appo = R[i]+R[j]-dist(pos,i,j)
        if(appo > 0):
            Rho[i][j] = appo
            
Noise = np.zeros((N,N), dtype=float)
for i in range(N):
    for j in range(i+1,N):
        Noise[i][j] = Rho[i][j]*Rho[i][j]*math.pi

In [ ]:
GRe=nx.Graph();
for i in range(N):
    GRe.add_node(i)
for i in range(N):
    for j in range(i+1,N):
        if(Rho[i][j] > 0):
            GRe.add_edge(i,j)
nx.draw(GRe,pos=pos,with_labels = True)

In [ ]:
q = Array.create('q', shape=N, vartype='BINARY')
H = 0;

A=np.zeros(N, dtype=float)
B=np.zeros((N,N), dtype=float)

alpha = 1

F = 10
beta = 0

L= np.zeros(N)

L[0]=1
L[12]=1
L[8]=1

gamma = 1

for i in range(N):
    A[i]= REPLACE_ME
    for j in range(N):
        B[i][j]= REPLACE_ME


for i in range(N):
    H = H + A[i]*q[i];
    for j in range(i+1,N):
        H = H + B[i][j]*q[i]*q[j];

model = H.compile()
bqm = model.to_bqm()

In [ ]:
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10000)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.sample);

In [ ]:
sols=[]
for samp in decoded_samples:
    res=[]
    for i in range(N):
        key='q['+str(i)+']';
        res.append(samp.sample[key])
    sols.append(res)
sol=[]
for i in range(N):
    key='q['+str(i)+']';
    sol.append(best_sample.sample[key])
print(sol)

In [ ]:
GSol =nx.Graph();
pos_sol = []
A_sol = []
for i in range(N):
    if(sol[i] > 0):
        GSol.add_node(i)
        pos_sol.append(pos[i])
        A_sol.append(Area[i])
nx.draw(GSol,pos=pos,with_labels = True)
ax = plt.gca()
for i in range(len(pos_sol)):
    center = pos_sol[i]
    c = plt.Circle(center, A_sol[i])
    ax.add_patch(c)
plt.axis('equal')
plt.show()

In [ ]:
nx.draw(GR,pos=pos,with_labels = True)
ax = plt.gca()
for i in range(N):
    center = pos[i]
    c = plt.Circle(center, Area[i])
    ax.add_patch(c)
plt.axis('equal')
plt.show()

In [ ]:
nx.draw(GSol,pos=pos,with_labels = True)
ax = plt.gca()
for i in range(len(pos_sol)):
    center = pos_sol[i]
    c = plt.Circle(center, A_sol[i])
    ax.add_patch(c)
plt.axis('equal')
plt.show()

In [ ]:
vals=[]
Areas=[]
Interf=[]
for ssol in sols:
    Hs = 0;
    As = 0;
    Is = 0;
    for i in range(N):
        Hs = Hs + A[i]*ssol[i];
        As = As + Area[i]*ssol[i];
        for j in range(i+1,N):
            Hs = Hs + B[i][j]*ssol[i]*ssol[j];
            Is = Is + Noise[i][j]*ssol[i]*ssol[j];
        
    vals.append(Hs)
    Areas.append(As)
    Interf.append(Is)
    
histogram = Counter(vals).most_common(3)
print(histogram)
histA = Counter(Areas).most_common(3)
print(histA)
histI = Counter(Interf).most_common(3)
print(histI)

Hs = 0;
As = 0;
Is = 0;

for i in range(N):
    Hs = Hs + A[i]*sol[i];
    As = As + Area[i]*ssol[i];
    for j in range(i+1,N):
        Hs = Hs + B[i][j]*sol[i]*sol[j];
        Is = Is + Noise[i][j]*sol[i]*sol[j];
        
print(Hs)
print(As)
print(Is)